In [1]:
import gradio as gr
import os
import time
from crewai import Agent, Task, Crew, Process
from crewai_tools import ScrapeWebsiteTool, SerperDevTool
from crewai_tools import tool
from datetime import datetime

#from docx import Document
from io import BytesIO
import base64

import numpy as np
import pandas as pd
from IPython.display import Markdown, display

from dotenv import load_dotenv
#load_dotenv("./.env")
load_dotenv("/home/jovyan/Albert/.env")

os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")
os.environ["SLIP_AUTH_URL"] = os.environ.get("SLIP_AUTH_URL")
os.environ["SLIP_USERNAME"] = os.environ.get("SLIP_USERNAME")
os.environ["SLIP_PASSWORD"] =  os.environ.get("SLIP_PASSWORD")
os.environ["SLIP_CLIENT_ID"]= os.environ.get("SLIP_CLIENT_ID")
os.environ["SLIP_APP_NAME"]=os.environ.get("SLIP_APP_NAME")
os.environ["SERPER_API_KEY"] = "25d43901c968b0f8830387b7ed72231673428b9c"

/home/jovyan/envs/llm_agents/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from llm_utils import get_llm
llm = get_llm('gpt-4o-0806',temperature=0)

In [3]:
df=pd.read_csv('/home/jovyan/data/LLM/EMR_LLM_data_combo_2204mcids.csv')

/home/jovyan/tmp/ipykernel_214257/4283731714.py:1: DtypeWarning: Columns (4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('/home/jovyan/data/LLM/EMR_LLM_data_combo_2204mcids.csv')


In [4]:
df=df[~df['type'].isin(['EMR_Note'])]

In [5]:
df['value']=np.round(df['value'],2)

In [6]:
df.columns = df.columns.str.strip()

In [7]:
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [8]:
# Define the new basic member information agent
basic_info_agent = Agent(
    role="Basic Member Information Provider",
    goal="Display the member ID, age, and gender for the specified member",
    backstory="You are a data retrieval specialist responsible for quickly and accurately providing basic member information.",
    verbose=True,
    allow_delegation=False,
    llm=llm  
)

In [9]:
diagnosis_agent = Agent(
    role="Diagnosis Analyst",
    goal="Provide a comprehensive analysis of the patient's diagnostic history",
    backstory="You are an expert in medical diagnostics with years of experience in analyzing patient histories.",
    verbose=True,
    allow_delegation=False,
    llm=llm  
)

procedure_agent = Agent(
    role="Procedure Analyst",
    goal="Analyze and summarize all medical procedures the patient has undergone",
    backstory="You are a medical procedure specialist with deep knowledge of various medical interventions.",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

medication_agent = Agent(
    role="Medication Analyst",
    goal="Provide a detailed overview of the patient's medication history",
    backstory="You are a clinical pharmacologist with expertise in analyzing medication regimens and their effects.",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

lab_test_agent = Agent(
    role="Lab Test Analyst",
    goal="Analyze and interpret all laboratory tests the patient has undergone",
    backstory="You are an expert in analyzing laboratory test codes and results. Your summaries are renowned for their depth, clarity, and insightful interpretation of lab test patterns and their clinical significance.",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

In [10]:
# Define the prompt for basic member information retrieval
BASIC_INFO_PROMPT = """
Retrieve and display the following information for the member:
1. Member ID: {member_id}
2. Age: {age}
3. Gender: {gender}


Example :
Member Information

Member ID: 10
Age: 35
Gender: Male

Format the output like the top in markdown format.

Note: Display "Female" if the value of gender is 0, "Male" if the value of gender is 1, and "Unknown" for any other value.
"""

In [11]:
DIAG_SUMMARY_PROMPT = """
Task Description
Your task is to summarize the following 'Patient Diagnosis Code History' and nothing else. The summary should be thorough, capturing ALL diagnoses present in the data.

Input Data
Patient Diagnosis Code History:
{diagnosis_history}

Output Format
Generate the summary in the following markdown format:

Diagnosis Summary:

Diagnosis Category 1:
- Diagnosis 2 (code2) in [dos1, dos2, dos3]
- Diagnosis 1 (code1) in [dos1, dos2, dos3, dos4, dos5]

Diagnosis Category 2:
- Diagnosis 6 (code6) in [dos1, dos2, dos3]
- Diagnosis 4 (code4) in [dos1]

Diagnosis Category 3: 
- Diagnosis 3 (code3) in [dos1, dos2, dos3, dos4, dos5]
- Diagnosis 5 (code5) in [dos1, dos2, dos3, dos4, dos5, dos6]

Rules and Guidelines

1.Only include diagnoses and dates explicitly present in the provided input data.
2.Do not infer or add any information not directly stated in the input data.
3.Each diagnosis should appear only once, with all associated dates listed chronologically.
4.Prioritize more serious and then recent disease categories to the top.
5.Within each category, prioritize more serious and recent diseases to the top.
6.If a disease does not fit into a designated category, list it as a stand-alone item.
7.Ensure ALL diagnoses from the input data are included in the summary.

Special Case
If there are no diagnoses in the patient's history, output only:
"There are no diagnoses observed in the patient's history."

Diagnosis Summary:
""".strip()

In [12]:
PROCEDURE_PROMPT = """

Task Description
Your task is to summarize the following 'Patient Procedure Code History' and nothing else. The summary should capture ALL procedures in the data.

Input Data
Patient Procedure Code History:
{procedure_history}

Output Format
Generate the summary in the following markdown format:

Procedure Summary:

Procedure Category 1:
- Proc 2 (code2) in [dos1, dos2, dos3]
- Proc 1 (code1) in [dos1, dos2, dos3, dos4, dos5]

Procedure Category 2:
- Proc 6 (code6) in [dos1, dos2, dos3, dos4, dos5]
- Proc 4 (code4) in [dos1, dos2, dos3]

Procedure Category 3: 
- Proc 3 (code3) in [dos1, dos2, dos3]
- Proc 5 (code5) in [dos1, dos2]

Rules and Guidelines

1.Only include procedures and dates explicitly present in the provided input data.
2.Do not infer or add any information not directly stated in the input data.
3.Each procedure should appear only once, with all associated dates listed chronologically.
4.Prioritize more serious and then recent procedure categories to the top.
5.Within each category, prioritize more serious and recent procedures to the top.
6.If a procedure does not fit into a designated category, list it as a stand-alone item.
7.Ensure ALL procedures from the input data are included in the summary.

Special Case
If there are no procedures in the patient's history, output only:
"There were no procedures performed in the patient's history."

Procedure Summary:
""".strip()


In [13]:
MEDICATION_PROMPT = """
Task Description
Your task is to summarize the following 'Patient RX Code History' and nothing else. The summary should account for ALL medications in the data.

Input Data
Patient RX Code History:
{medication_history}

Output Format
Generate the summary in the following markdown format:

Prescription Medication Summary:

Rx Category 1:
- Rx 2 (code2) in [dos1, dos2, dos3]
- Rx 1 (code1) in [dos1, dos2, dos3, dos4]

Rx Category 2:
- Rx 6 (code6) in [dos1, dos2, dos3, dos4, dos5]
- Rx 4 (code4) in [dos1, dos2, dos3]

Rx Category 3: 
- Rx 3 (code3) in [dos1, dos2, dos3, dos4, dos5]
- Rx 5 (code5) in [dos1, dos2, dos3, dos4, dos5, dos6]

Rules and Guidelines

1.Only include medications and dates explicitly present in the provided input data.
2.Do not infer or add any information not directly stated in the input data.
3.Each medication should appear only once, with all associated dates listed chronologically.
4.Prioritize more serious and then recent medication categories to the top.
5.Within each category, prioritize more serious and recent medications to the top.
6.If a medication does not fit into a designated category, list it as a stand-alone item.
7.Ensure ALL medications from the input data are included in the summary.

Special Case
If there are no medications in the patient's history, output only:
"There were no medications prescribed in the patient's history."


Prescription Medication Summary :
""".strip()


In [14]:
LAB_TEST_PROMPT = """
Task Description
Your task is to summarize the following 'Patient Lab Code History' and nothing else. The summary should capture ALL lab tests in the data.

Input Data
Patient Lab Code History:
{lab_test_history}

Output Format
Generate the summary in the following markdown format:

Lab Summary:

Lab Category 1:
- Complete Lab Test Name 2 (code2) in [dos1:value1,dos2:value2,dos3:value3]
- Complete Lab Test Name 1 (code1) in [dos1:value1,dos2:value2,dos3:value3]

Lab Category 2:
- Complete Lab Test Name 6 (code6) in [dos1:value1,dos2:value2,dos3:value3]  
- Complete Lab Test Name 4 (code4) in [dos1:value1,dos2:value2,dos3:value3]  

Lab Category 3: 
- Complete Lab Test Name 3 (code3) in [dos1:value1,dos2:value2,dos3:value3,dos4:value4]
- Complete Lab Test Name 5 (code5) in [dos1:value1,dos2:value2,dos3:value3,dos4:value4,dos5:value5]

Rules and Guidelines

1.Only include lab tests, dates, and results explicitly present in the provided input data.
2.Do not infer or add any information not directly stated in the input data.
3.Each lab test should appear only once, with all associated dates and results listed chronologically.
4.Prioritize more serious and then recent lab categories to the top.
5.Within each category, prioritize more serious and recent lab tests to the top.
6.If a lab test does not fit into a designated category, list it as a stand-alone item.
7.Ensure ALL lab tests from the input data are included in the summary.

Special Case
If there are no lab tests in the patient's history, output only:
"There were no lab tests conducted in the patient's history."

Lab Summary:
""".strip()

In [15]:
# Tasks for each agent
# Create a task for basic member information retrieval
basic_info_task = Task(
    description=BASIC_INFO_PROMPT,
    agent=basic_info_agent,
    expected_output="A concise display of the member's ID, age, and gender."
)

diagnosis_task = Task(
    description=DIAG_SUMMARY_PROMPT,
    agent=diagnosis_agent,
    expected_output="A comprehensive analysis of the patient's diagnostic history, or a statement indicating no diagnoses if applicable"
)

procedure_task = Task(
    description=PROCEDURE_PROMPT,
    agent=procedure_agent,
    expected_output="A detailed summary of all medical procedures the patient has undergone, or a statement indicating no procedures if applicable"
)

medication_task = Task(
    description=MEDICATION_PROMPT,
    agent=medication_agent,
    expected_output="A comprehensive summary of the patient's medication history, or a statement indicating no medications if applicable"
)

lab_test_task = Task(
    description=LAB_TEST_PROMPT,
    agent=lab_test_agent,
    expected_output="A detailed analysis of all lab tests with results the patient has undergone, or a statement indicating no lab tests if applicable"
)


In [16]:
past_condition_agent = Agent(
    role="Past Medical Condition Analyst",
    goal="Analyze and summarize the patient's past medical conditions based on their medical history, providing specific evidence including lab tests, codes, and impact on current health status.",
    tools=[scrape_tool, search_tool],
    verbose=True,
    llm=llm,
    backstory=(
        "As a specialized medical analyst with a focus on historical health analysis, you excel at identifying "
        "and summarizing past medical conditions based on a patient's medical history. Your expertise lies in "
        "recognizing patterns in historical data, understanding disease progression, and analyzing how past "
        "conditions impact current health status. You're skilled at providing evidence-based summaries, "
        "including detailed diagnosis, procedure, medication codes and lab test results and explaining the relevance of past conditions to the patient's overall health trajectory."
    )
)

past_condition_task = Task(
    description=(
        "1. Review the outputs from the basic info, diagnosis, procedure, medication, and lab test agents.\n"
        "2. Identify and list all past medical conditions based on the patient's medical history.\n"
        "3. Prioritize the past conditions based on their significance and impact on the patient's current health status.\n"
        "4. Structure the output as follows:\n"
        "   a. Start with a 'Prioritized List of Past Medical Conditions'.\n"
        "   b. After the list, add the note: 'See below for a summary and detailed analysis of each condition.'\n"
        "   c. Create a concise summary (around 200 words) of the patient's past health issues, titled 'Summary of Past Health Issues', highlighting the most significant conditions.\n"
        "   d. Follow with a 'Detailed Analysis of Each Past Medical Condition' section.\n"
        "5. For each past condition:\n"
        "   a. State the condition clearly.\n"
        "   b. Provide a detailed timeline including:\n"
        "      - Date(s) when the condition was diagnosed\n"
        "      - Date(s) of any related procedures\n"
        "      - Date(s) and duration of prescribed medications\n"
        "      - Date(s) of relevant lab tests\n"
        "   c. For each event (diagnosis, procedure, medication, lab test), clearly display:\n"
        "      - The exact date (or most specific timeframe if exact date is unavailable)\n"
        "      - The associated medical code\n"
        "      - The full name or description of the code\n"
        "   d. Describe the treatments or procedures undertaken for the condition, with specific dates, codes, and names.\n"
        "   e. List medications prescribed, including start dates, duration of treatment, codes, and full names.\n"
        "   f. Provide lab test evidence:\n"
        "      - Name of the lab test(s) related to the condition\n"
        "      - Date(s) when each test was conducted\n"
        "      - Results of each test, including numerical values rounded to two decimal places and reference ranges where applicable\n"
        "      - Brief interpretation of the test results in the context of the condition\n"
        "   g. Discuss the impact of this past condition on the patient's current health status.\n"
        "   h. Provide any additional specific evidence from the patient's records that supports this analysis, including dates, codes, and names for all events.\n"
        "6. Ensure that all dates, codes, and full names/descriptions for diagnoses, procedures, treatments, medications, and lab tests are clearly stated for each condition.\n"
        "7. If exact dates are not available for any event, clearly state this and provide the most specific timeframe possible based on the available data.\n"
        "8. For each lab test mentioned, ensure you provide context on how it relates to the condition and what the results (rounded to two decimal places) indicate about the patient's health status at that time."
    ),
    expected_output=(
        "A detailed analysis of the patient's past medical conditions, structured as follows:\n"
        "1. Prioritized List of Past Medical Conditions\n"
        "2. Note: See below for a summary and detailed analysis of each condition.\n"
        "3. Summary of Past Health Issues (about 200 words)\n"
        "4. Detailed Analysis of Each Past Medical Condition:\n"
        "   - Clear identification of the condition\n"
        "   - Detailed timeline including dates, codes, and full names/descriptions for diagnoses, procedures, medication prescriptions, and relevant lab tests\n"
        "   - Description of treatments or procedures with specific dates, codes, and names\n"
        "   - List of medications with start dates, duration, codes, and full names\n"
        "   - Lab test evidence, including:\n"
        "     * Name of relevant lab tests\n"
        "     * Dates of tests\n"
        "     * Test results with numerical values rounded to two decimal places and reference ranges\n"
        "     * Brief interpretation of results\n"
        "   - Impact on current health status\n"
        "   - Any additional specific evidence from patient records, including dates, codes, and names for all events\n"
        "Note: All dates should be clearly stated. If exact dates are unavailable, the most specific timeframe possible should be provided. "
        "Ensure that the relevance of each lab test to the condition is explained, and the interpretation of results (rounded to two decimal places) is provided in the context of the patient's health status."
    ),
    agent=past_condition_agent,
    context=[basic_info_task, diagnosis_task, procedure_task, medication_task, lab_test_task]
)

In [17]:
future_condition_agent = Agent(
    role="Future Medical Condition Analyst",
    goal="Analyze and predict potential future medical conditions for the patient based on their medical history and current status, providing specific evidence and risk factors, including normal ranges for all lab tests mentioned.",
    tools=[scrape_tool, search_tool],
    verbose=True,
    llm=llm,
    backstory=(
        "As a specialized medical analyst with a focus on predictive health analysis, you excel at identifying "
        "potential future medical conditions based on a patient's medical history and current status. Your expertise "
        "lies in recognizing risk factors, understanding disease progression, and analyzing trends in patient data "
        "to forecast possible health issues. You're skilled at providing evidence-based predictions and suggesting "
        "preventive measures to mitigate future health risks. You always provide normal ranges for any lab tests mentioned."
    )
)

future_condition_task = Task(
    description=(
        "1. Review the outputs from the basic info, diagnosis, procedure, medication, lab test, and summary agents, as well as the past and present condition summaries.\n"
        "2. Identify potential future medical conditions based on the patient's medical history, current status, and risk factors.\n"
        "3. Exclude any conditions that have already occurred in the patient's past medical history.\n"
        "4. Prioritize the potential future conditions based on their likelihood and potential impact on the patient's health.\n"
        "5. Structure the output as follows:\n"
        "   a. Start with a 'Prioritized List of Potential Future Conditions'.\n"
        "   b. After the list, add the note: 'See below for a summary and detailed analysis of each condition.'\n"
        "   c. Create a concise summary (around 200 words) of the patient's potential future health risks, titled 'Summary of Potential Future Health Risks'.\n"
        "   d. Follow with a 'Detailed Analysis of Each Potential Future Condition' section.\n"
        "6. For each potential future condition in the detailed analysis:\n"
        "   a. State the potential condition clearly.\n"
        "   b. Explain the risk factors present in the patient's history that suggest this potential condition.\n"
        "   c. Provide specific evidence from the patient's records that supports this prediction:\n"
        "      - For any medications or procedures related to the possibility of the condition, mention:\n"
        "        * The name of the medication or procedure\n"
        "        * The specific date when it occurred\n"
        "        * The associated medical code\n"
        "        * The full description of the code\n"
        "      - For ANY lab test mentioned:\n"
        "        * ALWAYS include the normal range for that test\n"
        "        * Round all lab test results to two decimal places\n"
        "        * Include the date when the test was conducted\n"
        "   d. When mentioning any medical code (diagnosis, procedure, medication), always display:\n"
        "      * The proper name of the code\n"
        "      * The code description\n"
        "      * The date when it occurred\n"
        "   e. Estimate the likelihood or timeframe for this condition to develop, if possible.\n"
        "   f. Suggest preventive measures or monitoring strategies to mitigate this future health risk."
    ),
    expected_output=(
        "A detailed analysis of the patient's potential future medical conditions, structured as follows:\n"
        "1. Prioritized List of Potential Future Conditions\n"
        "2. Note directing to summary and detailed analysis\n"
        "3. Summary of Potential Future Health Risks (about 200 words)\n"
        "4. Detailed Analysis of Each Potential Future Condition, including for each:\n"
        "   - Clear identification of the condition\n"
        "   - Explanation of risk factors\n"
        "   - Specific evidence supporting the prediction, including:\n"
        "     * For medications and procedures: name, date of occurrence, medical code, and full code description\n"
        "     * For ALL lab tests: normal ranges, results rounded to two decimal places, and test dates\n"
        "   - For any medical code mentioned: proper name, description, and occurrence date\n"
        "   - Estimated likelihood or timeframe for condition development, if possible\n"
        "   - Suggested preventive measures or monitoring strategies"
    ),
    agent=future_condition_agent,
    context=[basic_info_task, diagnosis_task, procedure_task, medication_task, lab_test_task, past_condition_task]
)

In [18]:
question_answer_generator_agent = Agent(
    role="Medical Question and Answer Generator",
    goal="Generate three important questions with answers to help patients discuss their condition with doctors",
    tools=[search_tool, scrape_tool],
    verbose=True,
    llm=llm,
    backstory=(
        "You are an experienced medical professional with expertise in patient education and doctor-patient communication. "
        "Your role is to generate insightful questions and provide informative answers that will help patients have more "
        "productive discussions with their doctors about their condition, treatment options, and overall health management."
    )
)

question_answer_generator_task = Task(
    description=(
        "Based on the patient's medical information, generate three important questions with answers to help the patient "
        "discuss their condition with their doctor. Follow these steps:\n"
        "1. Review the patient's diagnosis history, lab results, medication history, procedure history, "
        "past medical conditions, and potential future conditions.\n"
        "2. Identify the most critical aspects of the patient's health status and medical history.\n"
        "3. Formulate three specific, targeted questions that address:\n"
        "   a. Understanding their current condition and its implications\n"
        "   b. Treatment options or management strategies\n"
        "   c. Preventive measures or lifestyle changes\n"
        "4. For each question, provide a clear and informative answer based on the patient's specific case and general medical knowledge.\n"
        "5. Ensure each question and answer pair is relevant to the patient's case and would provide valuable information for a doctor visit.\n"
        "6. Format the output as a markdown list with questions and answers on separate lines.\n"
        "7. After each question-answer pair, briefly explain why this information is important for this patient's case.\n"
    ),
    agent=question_answer_generator_agent,
    context=[diagnosis_task, lab_test_task, medication_task, procedure_task, past_condition_task, future_condition_task],
    expected_output=(
        "A markdown-formatted list of three important medical questions with answers, specific to the patient's case, "
        "each followed by a brief explanation of its relevance. Questions and answers should be on separate lines."
    )
)

In [21]:
import matplotlib.pyplot as plt
import io
from PIL import Image
from typing import List, Dict, Union

import os
import matplotlib.pyplot as plt
from datetime import datetime
from typing import List, Dict, Union

@tool
def plot_lab_test(test_name: str, data: List[Dict[str, Union[str, float]]], config: dict = None):
    """
    Generate a graph of lab test results over time and return a filename.
    
    Parameters:
    test_name (str): The name of the lab test, used for the graph title and y-axis label.
    data (List[Dict[str, Union[str, float]]]): A list of dictionaries, where each dictionary
        contains 'date' (str) and 'value' (float) keys representing a single test result.
    config (dict, optional): Configuration options for plot customization.
    
    Returns:
    str: Filename of the saved plot
    """
    # Set default configuration
    default_config = {
        'figsize': (10, 6),
        'marker': 'o',
        'rotation': 45,
        'color': 'blue',
        'line_style': '-'
    }
    
    # Override default config with user-provided config
    if config is not None:
        default_config.update(config)
    
    # Extract dates and values
    dates = [point['date'] for point in data]
    values = [point['value'] for point in data]
    
    # Create the plot
    plt.figure(figsize=default_config['figsize'])
    plt.plot(
        dates, 
        values, 
        marker=default_config['marker'], 
        color=default_config['color'],
        linestyle=default_config['line_style']
    )
    plt.title(f'{test_name} Test Results Over Time')
    plt.xlabel('Date')
    plt.ylabel(f'{test_name} Value')
    plt.xticks(rotation=default_config['rotation'])
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    
    # Create a directory to save plots if it doesn't exist
    os.makedirs('saved_plots', exist_ok=True)
    
    # Generate a unique filename using the test name and current timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = os.path.join('saved_plots', f"{test_name.replace(' ', '_')}_{timestamp}.png")
    
    # Save the plot as an image file
    plt.savefig(filename)
    
    # Clear the current figure to free up memory
    plt.clf()
    plt.close()
    
    return filename

In [22]:
# Define the Intelligent Medical Chatbot Agent
intelligent_medical_chatbot = Agent(
    role="Smart Medical Expert",
    goal="Provide accurate and helpful responses to user questions about medical conditions, interpret medical codes, and generate visualizations of lab test results when requested",
    tools=[search_tool, scrape_tool, plot_lab_test],  
    llm=llm, 
    backstory=(
        "You are an AI-powered medical expert chatbot with extensive knowledge of medical codes and the ability to analyze and interpret medical records. "
        "You can answer both patient-specific and general medical queries, leveraging your access to patient information including basic details, diagnosis history, "
        "procedure history, medication history, lab test results, past medical conditions, and potential future conditions. "
        "You are allowed to make multiple calls (either together or in sequence)."
        "You're equipped with a lab visualization tool for generating graphs of lab test results. "
        "Your responses are based on your comprehensive medical knowledge and the patient data provided."
    )
)

intelligent_medical_chatbot_task = Task(
    description=(
        "As a smart medical expert, your task is to answer '{user_question}' accurately and helpfully. Follow these guidelines:\n"
        "1. Analyze the user's question to determine the type of information needed.\n"
        "2. If the question requires patient-specific information, review and use the provided patient data.\n"
        "3. Use your extensive medical knowledge to interpret and explain medical information.\n"
        "4. Interpret medical codes using your expertise when required.\n"
        "5. For lab test visualization:\n"
        "   - If explicitly requested or if it would enhance your explanation, use the plot_lab_test tool.\n"
        "   - Provide the exact lab_code name when calling the tool.\n"
        "   - The tool will return a filename (e.g., 'saved_plots/lab_code_timestamp.png').\n"
        "   - Include this filename in your response text.\n"
        "   - Do not attempt to embed the image yourself; the chat function will handle this.\n"
        "6. You can use the plot_lab_test tool multiple times if needed for different lab tests.\n"
        "7. Format your response clearly:\n"
        "   - Provide a direct answer to the question.\n"
        "   - Include relevant details from patient information or your medical knowledge.\n"
        "   - If you've generated a graph, mention the filename in your text response.\n"
        "   - Provide a brief interpretation of the graph.\n"
        "8. If the question is outside your scope or requires immediate medical attention, advise consulting a healthcare professional.\n"
        "9. Maintain patient confidentiality at all times.\n"
    ),
    agent=intelligent_medical_chatbot,
    context=[basic_info_task,diagnosis_task,procedure_task,medication_task,lab_test_task,past_condition_task,future_condition_task],
    expected_output=(
        "A clear, informative, and accurate response addressing the user's question. "
        "If patient-specific information is used, clearly reference the sources. "
        "If a graph was generated, the filename should be mentioned in the text response. "
        "The response should combine textual explanation with references to generated graphs when appropriate."
    )
)

In [23]:
# Define the Evaluation Agent 
evaluation_agent = Agent(
    role="Result Validator", 
    goal="Evaluate the correctness of the generated summaries by comparing them with the original patient data and report only discrepancies.", 
    backstory="You are an expert in medical data validation. Your task is to ensure that the summaries generated by other agents match the raw patient data accurately, focusing on identifying and reporting any discrepancies.", 
    verbose=True,
    allow_delegation=False,
    llm=llm
)

EVALUATION_PROMPT = """
Role and Goal
You are an expert medical data validator tasked with ensuring the accuracy of generated patient summaries.
Your primary goal is to meticulously compare the generated summaries {generated_summary} with the original {patient_data} and 
report only genuine discrepancies.

Instructions
Carefully analyze the following outputs generated by other agents by comparing them with the original {patient_data}. 
It is crucial that you only report actual discrepancies. 
Follow these steps for each type of data (Diagnosis, Procedure, Medication, Lab Test, Past Medical Condition, Future Medical Condition):

1. Thoroughly compare the summary with the raw data.
2. Check for any incorrectly represented or added information in the summaries.
3. Before reporting any discrepancy:
   a. Double-check that the information is truly inconsistent with the original data.
   b. Ensure you're not overlooking any data in the original patient record.
   c. Verify that you're not misinterpreting any information.
4. Only report a discrepancy if you are absolutely certain it exists after thorough verification.
5. If no discrepancies are found for a particular type of data, do not mention it in your report.

Input Data
Original Patient Data: {patient_data}
Generated Summary: {generated_summary}

Output Format

Generate the report in the following markdown format:

Evaluation Report:
[Only list verified discrepancies. Omit any section where no discrepancies are found.]

Diagnosis Discrepancies:

[List specific discrepancies, providing evidence from both the summary and original data]

Procedure Discrepancies:

[List specific discrepancies, providing evidence from both the summary and original data]

Medication Discrepancies:

[List specific discrepancies, providing evidence from both the summary and original data]

Lab Test Discrepancies:

[List specific discrepancies, providing evidence from both the summary and original data]

Past Medical Condition Discrepancies:

[List specific discrepancies, providing evidence from both the summary and original data]

Future Medical Condition Discrepancies:

[List specific discrepancies, providing evidence from both the summary and original data]

If no discrepancies are found in any category, simply state: "No discrepancies found. All generated summaries accurately match the original patient data."
"""

In [24]:
evaluation_task = Task(
    description=EVALUATION_PROMPT, 
    agent=evaluation_agent, 
    expected_output="A detailed evaluation report highlighting any discrepancies between the raw patient data and the generated summaries. If no discrepancies are found, a statement confirming the accuracy of the summaries."
)

In [25]:
def process_member_data(member_id):
    df_member = df[df['id'] == member_id]
    
    if df_member.empty:
        return None, None, None, None
    
     #extract basic information 
    df_age_gender=df_member[(df_member['type']=='age') | (df_member['type']=='gender')].reset_index(drop=True)
    age=df_age_gender.loc[0,'value']
    gender=df_age_gender.loc[1,'value']
    
    df_diag = df_member[df_member['type'] == 'diag']
    df_diag = df_diag[['id', 'dos', 'code', 'codeDesc', 'type']]
    df_proc = df_member[df_member['type'] == 'proc']
    df_proc = df_proc[['id', 'dos', 'code', 'codeDesc', 'type']]
    df_rx = df_member[df_member['type'] == 'rx']
    df_rx = df_rx[['id', 'dos', 'code', 'codeDesc', 'type']]
    df_lab = df_member[df_member['type'] == 'lab']
    
    basic_info_crew = Crew(agents=[basic_info_agent], tasks=[basic_info_task], verbose=True)
    basic_info_result = basic_info_crew.kickoff({"member_id": member_id,"age":age,"gender":gender})
    
    diagnosis_crew = Crew(agents=[diagnosis_agent], tasks=[diagnosis_task], verbose=True)
    diag_result = diagnosis_crew.kickoff({"diagnosis_history": df_diag.to_string()})
    
    procedure_crew = Crew(agents=[procedure_agent], tasks=[procedure_task], verbose=True)
    proc_result = procedure_crew.kickoff({"procedure_history": df_proc.to_string()})
    
    medication_crew = Crew(agents=[medication_agent], tasks=[medication_task], verbose=True)
    med_result = medication_crew.kickoff({"medication_history": df_rx.to_string()})
    
    lab_test_crew = Crew(agents=[lab_test_agent], tasks=[lab_test_task], verbose=True)
    lab_result = lab_test_crew.kickoff({"lab_test_history": df_lab.to_string()})
    
    past_condition_crew = Crew(agents=[past_condition_agent], tasks=[past_condition_task], verbose=True)
    past_condition_result = past_condition_crew.kickoff()
    
    future_condition_crew = Crew(agents=[future_condition_agent], tasks=[future_condition_task], verbose=True)
    future_condition_result = future_condition_crew.kickoff()
    
    question_answer_generator_crew = Crew(agents=[question_answer_generator_agent], tasks=[question_answer_generator_task], verbose=True)
    question_answer_generator_result = question_answer_generator_crew.kickoff()
    
    return basic_info_result,diag_result, proc_result, med_result, lab_result,past_condition_result,future_condition_result,question_answer_generator_result

In [26]:
def gradio_interface(member_id): 
    try:
        member_id = int(member_id) 
    except ValueError:
        return "Invalid member ID. Please enter a number.", "", "", "", "", "", "", "", "", ""
    
    # Process the member data and get results from other agents 
    results = process_member_data(member_id) 
    if results[0] is None:
        return "No data found for this member ID.", "", "", "", "", "", "", "", "", "" 
    
    evaluation_crew = Crew(agents=[evaluation_agent], tasks=[evaluation_task], verbose=True)
    df_member = df[df['id'] == member_id]
    
    # Now process the result-checking agent 
    evaluation_result = evaluation_crew.kickoff({ "patient_data": df_member.to_string(), 
                                                 "generated_summary": { "basic_info": results[0], "diagnosis": results[1], "procedure": results[2], "medication": results[3], "lab_test": results[4],"past_medical_condition": results[5],"future_medical_condition": results[6]}}) 
    
    return (*results, evaluation_result) 

In [27]:
import re
import base64
from PIL import Image
import io

def chat(message, history):
    med_chatbot_crew = Crew(agents=[intelligent_medical_chatbot], tasks=[intelligent_medical_chatbot_task], verbose=True)
    
    response = med_chatbot_crew.kickoff({"user_question": message, "History": history})
    
    # Use regex to find all .png filenames in the saved_plots directory
    image_paths = re.findall(r'saved_plots/[^()]+\([^)]+\)_\d+_\d+\.png', response)
    print(image_paths)
    
    if image_paths:
        # Get the last image path (in case there are multiple)
        image_path = image_paths[-1]
        print(f"Image path: {image_path}")
        
        # Extract the text response by removing the image path
        text_response = response.replace(image_path, '').strip()
        
        try:
            with Image.open(image_path) as img:
                buffer = io.BytesIO()
                img.save(buffer, format="PNG")
                encoded_image = base64.b64encode(buffer.getvalue()).decode()
            
            full_response = f"{text_response}\n\nHere's the requested graph:\n\n![Lab Test Graph](data:image/png;base64,{encoded_image})"
            
            history.append((message, full_response))
        except Exception as e:
            error_response = f"An error occurred while processing the graph: {str(e)}"
            history.append((message, f"{text_response}\n\n{error_response}"))
    else:
        history.append((message, response))
    
    return "", history

In [28]:
def save_results(member_id, basic_info, diag, proc, med, lab, past_condition, future_condition, qa, evaluation):
    results = {
        "Member ID": member_id,
        "Basic Info": basic_info,
        "Diagnosis Summary": diag,
        "Procedure Summary": proc,
        "Medication Summary": med,
        "Lab Summary": lab,
        "Past Medical Conditions": past_condition,
        "Potential Future Conditions": future_condition,
        "Common Q&A": qa,
        "Evaluation Results": evaluation
    }
    
    filename = f"results_for_patient_ID{member_id}.md"
    with open(filename, 'w') as f:
        f.write(f"# Medical Record Results for Member ID: {member_id}\n\n")
        for key, value in results.items():
            f.write(f"## {key}\n\n{value}\n\n")
    
    return f"Results saved to {filename}"

In [29]:
# Create Gradio blocks
with gr.Blocks(title="Chat With Medical Records") as demo:
    gr.Markdown("# Chat With Medical Records")
    gr.Markdown("Enter a member ID to get a detailed analysis of the patient's medical history.")
    
    with gr.Row():
        member_id_input = gr.Textbox(label="Enter Member ID")
        submit_button = gr.Button("Submit")

    with gr.Tabs():
        with gr.TabItem("Member Info."):
            basic_info_output = gr.Markdown()
        with gr.TabItem("Diagnosis Summary"):
            diag_output = gr.Markdown()
        with gr.TabItem("Procedure Summary"):
            proc_output = gr.Markdown()
        with gr.TabItem("Medication Summary"):
            med_output = gr.Markdown()
        with gr.TabItem("Lab Summary"):
            lab_output = gr.Markdown()
        with gr.TabItem("Past Medical Conditions"):
            past_condition_output = gr.Markdown()
        with gr.TabItem("Potential Future Conditions"):
            future_condition_output = gr.Markdown()  
        with gr.TabItem("Ask doctor"):
            question_answer_generator_output = gr.Markdown()
        with gr.TabItem("Evaluation Results"): 
            evaluation_output = gr.Markdown() 
            
            
        save_button = gr.Button("Save Results")
        save_output = gr.Markdown()
      
        submit_button.click(
        fn=gradio_interface,
        inputs=member_id_input,
        outputs=[basic_info_output, diag_output, proc_output, med_output, lab_output, past_condition_output, future_condition_output,question_answer_generator_output,evaluation_output])
        
        save_button.click(
            fn=save_results,
            inputs=[member_id_input, basic_info_output, diag_output, proc_output, med_output, lab_output, past_condition_output, future_condition_output, question_answer_generator_output, evaluation_output],
            outputs=save_output
        )
    
    with gr.Tabs():
        with gr.TabItem("Medical Chatbot"):
            chatbot = gr.Chatbot()
            msg = gr.Textbox(label="Ask question")
            clear = gr.Button("Clear")

            msg.submit(chat, [msg, chatbot], [msg, chatbot])
            clear.click(lambda: None, None, chatbot, queue=False)

/home/jovyan/envs/llm_agents/lib/python3.11/site-packages/gradio/components/chatbot.py:223: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


In [30]:
demo.launch(share=True)

which: no node in (/home/jovyan/envs/llm_agents/bin:/miniconda3/condabin:/opt/mssql-tools/bin:/usr/local/apache-jmeter-5.4.3/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/miniconda3/bin:/usr/lib/spark:/usr/lib/spark/bin:/usr/lib/jvm/java-1.8.0/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin)


* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://c03ad747e6c67a9612.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


 [2024-11-21 19:25:00][DEBUG]: == Working Agent: Basic Member Information Provider
 [2024-11-21 19:25:00][INFO]: == Starting Task: 
Retrieve and display the following information for the member:
1. Member ID: 48
2. Age: 92.0
3. Gender: 0.0


Example :
Member Information

Member ID: 10
Age: 35
Gender: Male

Format the output like the top in markdown format.

Note: Display "Female" if the value of gender is 0, "Male" if the value of gender is 1, and "Unknown" for any other value.



> Entering new CrewAgentExecutor chain...


2024-11-21 19:25:02,417 - 140291380066048 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


I now can give a great answer  
Final Answer: 

```markdown
Member Information

Member ID: 48  
Age: 92  
Gender: Female
```

> Finished chain.
 [2024-11-21 19:25:02][DEBUG]: == [Basic Member Information Provider] Task output: ```markdown
Member Information

Member ID: 48  
Age: 92  
Gender: Female
```


 [2024-11-21 19:25:02][DEBUG]: == Working Agent: Diagnosis Analyst
 [2024-11-21 19:25:02][INFO]: == Starting Task: Task Description
Your task is to summarize the following 'Patient Diagnosis Code History' and nothing else. The summary should be thorough, capturing ALL diagnoses present in the data.

Input Data
Patient Diagnosis Code History:
        id         dos    code                                                                                                                                                     codeDesc  type
189945  48  2018-06-05    E039                                                                                                                              

...

The patient's medical history indicates several chronic conditions, including Alzheimer's disease, hypothyroidism, hyperlipidemia, and peripheral vascular disease, among others. These conditions require careful management and monitoring to prevent complications and improve quality of life. Based on this information, I will generate three important questions with answers to help the patient discuss their condition with their doctor.

Final Answer:

1. **Question:** What are the current treatment options for managing my Alzheimer's disease, and how can they help slow its progression?

   **Answer:** Current treatment options for Alzheimer's disease include medications such as cholinesterase inhibitors (e.g., donepezil, rivastigmine) and NMDA receptor antagonists (e.g., memantine). These medications can help manage symptoms by improving communication between nerve cells in the brain, potentially slowing cognitive decline. Additionally, non-pharmacological interventions like cognitive ther

2024-11-21 19:27:46,817 - 140291380066048 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


 [2024-11-21 19:27:46][DEBUG]: == Working Agent: Smart Medical Expert
 [2024-11-21 19:27:46][INFO]: == Starting Task: As a smart medical expert, your task is to answer 'plot a graph for Basophils/100 leukocytes (706-2) ' accurately and helpfully. Follow these guidelines:
1. Analyze the user's question to determine the type of information needed.
2. If the question requires patient-specific information, review and use the provided patient data.
3. Use your extensive medical knowledge to interpret and explain medical information.
4. Interpret medical codes using your expertise when required.
5. For lab test visualization:
   - If explicitly requested or if it would enhance your explanation, use the plot_lab_test tool.
   - Provide the exact lab_code name when calling the tool.
   - The tool will return a filename (e.g., 'saved_plots/lab_code_timestamp.png').
   - Include this filename in your response text.
   - Do not attempt to embed the image yourself; the chat function will handle th

2024-11-21 19:30:18,216 - 140291380066048 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


 [2024-11-21 19:30:18][DEBUG]: == Working Agent: Smart Medical Expert
 [2024-11-21 19:30:18][INFO]: == Starting Task: As a smart medical expert, your task is to answer 'plot a graoh for Monocytes (742-7) ' accurately and helpfully. Follow these guidelines:
1. Analyze the user's question to determine the type of information needed.
2. If the question requires patient-specific information, review and use the provided patient data.
3. Use your extensive medical knowledge to interpret and explain medical information.
4. Interpret medical codes using your expertise when required.
5. For lab test visualization:
   - If explicitly requested or if it would enhance your explanation, use the plot_lab_test tool.
   - Provide the exact lab_code name when calling the tool.
   - The tool will return a filename (e.g., 'saved_plots/lab_code_timestamp.png').
   - Include this filename in your response text.
   - Do not attempt to embed the image yourself; the chat function will handle this.
6. You can 